# Análisis de las imágenes

## Carga del Dataset
Vamos a definir una clase, que tome como base la clase Dataset de Pytorch, para poder cargar todo nuestro conjunto de imágenes de uno o varios canales con su máscara correspondiente.

In [3]:
import os

print("Vamos a cambiar el directorio de trabajo")

# Indicamos la ruta del directorio de trabajo
route = os.getcwd() + "/TFG/test/PNe_segmentation"
os.chdir(route)

current_directory = os.getcwd()
print(" El directorio actual es:", current_directory)

# Listamos el contenido del directorio
files = os.listdir(current_directory)
print(" Contenido del directorio actual:")
for file in files:
    print("\t",file)
    
# Listamos el contenido del directorio de las máscaras
# masks_directory = route+"TFG\\test\\PNe_segmentation\\masks"
# data_directory = route+"TFG\\test\\PNe_segmentation\\data"
## Ejecución en el CESGA Finisterrae III
masks_directory = route+"/masks"
data_directory = route+"/data"


Vamos a cambiar el directorio de trabajo
 El directorio actual es: /mnt/netapp2/Home_FT2/home/ulc/co/ela/TFG/test/PNe_segmentation
 Contenido del directorio actual:
	 train_models
	 data
	 create_dataset.ipynb
	 data_files_1c.csv
	 images
	 image_analysis.ipynb
	 masks
	 segmentation_no_supervisada.ipynb
	 segmentation_no_supervisada_2c.ipynb
	 segmentation_no_supervisada_pytorch.ipynb
	 segmentation_supervisada.ipynb
	 dataset_info.csv
	 data_files_1c_train.csv
	 historico_notebooks
	 data_files_1c_test.csv
	 segmentation_no_supervisada_pytorch_clean.ipynb
	 segmentation_supervisada_pytorch copy.ipynb
	 segmentation_supervisada_pytorch.ipynb
	 cesga
	 segmentation_supervisada_pytorch copy 2.ipynb
	 segmentation_supervisada_pytorch copy 3.ipynb


In [59]:
from pnebulae_torch.dataset import NebulaeDataset
from torchvision import transforms
import torch
import pandas as pd

# Creamos un objeto de la clase PNeDataset
transform_x = transforms.Compose([
                    # MinMaxNorm,
                    # TypicalImageNorm(factor = 1, substract=0),
                    # ApplyMorphology(operation = morphology.binary_opening, concat = True, footprint = morphology.disk(2)),
                    # ApplyMorphology(operation = morphology.area_opening, concat = True, area_threshold = 200, connectivity = 1),
                    # ApplyIntensityTransformation(transformation = exposure.equalize_hist, concat = True, nbins = 4096),
                    # ApplyIntensityTransformation(transformation = exposure.equalize_adapthist, concat = True, nbins = 640, kernel_size = 5),
                    # ApplyMorphology(operation = morphology.area_opening, concat = True, area_threshold = 200, connectivity = 1),
                    # ApplyFilter(filter = ndimage.gaussian_filter, concat = True, sigma = 5),
                    # transforms.ToTensor(),
                    transforms.Lambda(lambda x: torch.from_numpy(x.copy()).unsqueeze_(0)),
                    # CustomPad(target_size = (1984, 1984), fill_min=True, tensor_type=torch.Tensor.float)
                    ])

transform_y = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Lambda(lambda x: torch.Tensor.int(x)),
                    # CustomPad(target_size = (1984, 1984), fill = 0, tensor_type=torch.Tensor.int)
                    ])

df_train = pd.read_csv("data_files_1c_train_correct.csv")
dataset_train = NebulaeDataset(data_directory, masks_directory, df_train, transform = (transform_x, transform_y))

In [70]:
for i in range(len(dataset_train)):
    print(dataset_train[i][0].max())
    if dataset_train[i][0].max() == 71253.5859:
        print("El archivo es:", i)
        break

tensor(4574.5986)
tensor(365.0783)
tensor(279.1983)
tensor(1208.6595)
tensor(3825.2434)
tensor(7432.4995)
tensor(977.3033)
tensor(1752.2561)
tensor(626.5923)
tensor(1735.3855)
tensor(244.0661)
tensor(733.6297)
tensor(188.1891)
tensor(794.0305)
tensor(5461.5781)
tensor(589.0709)
tensor(3324.)
tensor(10123.4336)
tensor(4850.9463)
tensor(298.4151)
tensor(3765.7341)
tensor(69677.0547)
tensor(238.7206)
tensor(617.5674)
tensor(1554.6730)
tensor(2776.8040)
tensor(1081.0856)
tensor(16571.4668)
tensor(0.9997)
tensor(2164.5503)
tensor(366.0248)
tensor(3965.0193)
tensor(166.2244)
tensor(792.7939)
tensor(4685.3159)
tensor(1976.6223)
tensor(11650.4160)
tensor(550.4119)
tensor(1196.4901)
tensor(62289.0117)
tensor(442.4381)
tensor(125873.7500)
tensor(1083.5076)
tensor(2157.9941)
tensor(3283.0576)
tensor(71253.5859)
El archivo es: 45


In [74]:
min_values = []
max_values = []

for i in range(len(dataset_train)):
    min_values.append(dataset_train[i][0].min())
    max_values.append(dataset_train[i][0].max())

print("Min value:", min(min_values))
print("Max value:", max(max_values))

Min value: tensor(-88.9933)
Max value: tensor(125873.7500)
